In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Specify work folder
DATA_DIR = r'P:\XAIM Biolab\2nd sem CVDL/HAM10000/'   #SPECIFY YOURS

# Load metadata
num_rows_to_read = None  # To load the whole dataset     #Alternatively, make a titration to test it first by specifiying a number (for e.g "5" instead of None)
data = pd.read_csv(os.path.join(DATA_DIR, 'HAM10000_metadata.csv'), nrows=num_rows_to_read)
data['image_path'] = DATA_DIR + 'HAM10000_all_images/' + data['image_id'] + '.jpg'

# Mapping dictionary
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

# Add two more columns
data['cell_type'] = data['dx'].map(lesion_type_dict.get)
data['cell_type_idx'] = pd.Categorical(data['cell_type']).codes

# Split dataset into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define the Swin Transformer model architecture
class SwinTransformer(nn.Module):
    def __init__(self, num_classes):
        super(SwinTransformer, self).__init__()
        # Define the layers of the model
        self.conv = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.linear = nn.Linear(64 * 224 * 224, num_classes)  # Assuming input image size is 224x224
        self.dropout = nn.Dropout(0.5)  # Dropout layer with a dropout probability of 0.5

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)  # Flatten the feature map
        x = self.dropout(x)  # Apply dropout
        x = self.linear(x)
        return x

# Define the training dataset class
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['image_path']
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(self.data.iloc[idx]['cell_type_idx'], dtype=torch.long)  # Convert label to torch.long
        return img, label

# Define transformations for data augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Random horizontal flip for data augmentation
    transforms.RandomRotation(15),  # Random rotation of up to 15 degrees for data augmentation
    transforms.ToTensor(),
])

# Create datasets and data loaders
train_dataset = CustomDataset(train_data, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomDataset(test_data, transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
]))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Instantiate the Swin Transformer model
model = SwinTransformer(num_classes=7)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with validation
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        correct_train_predictions = 0
        total_train_predictions = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * images.size(0)

            _, predicted_train = torch.max(outputs, 1)
            correct_train_predictions += (predicted_train == labels).sum().item()
            total_train_predictions += labels.size(0)

        train_loss = running_train_loss / len(train_loader.dataset)
        train_accuracy = correct_train_predictions / total_train_predictions
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Validation
        model.eval()
        running_val_loss = 0.0
        correct_val_predictions = 0
        total_val_predictions = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item() * images.size(0)

                _, predicted_val = torch.max(outputs, 1)
                correct_val_predictions += (predicted_val == labels).sum().item()
                total_val_predictions += labels.size(0)

        val_loss = running_val_loss / len(val_loader.dataset)
        val_accuracy = correct_val_predictions / total_val_predictions
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Plot training and validation loss
    plt.plot(train_losses, label='Training Loss', color='red')
    plt.plot(val_losses, label='Validation Loss', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()

    # Plot training and validation accuracy
    plt.plot(train_accuracies, label='Training Accuracy', color='purple')
    plt.plot(val_accuracies, label='Validation Accuracy', color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    plt.show()


In [ ]:
# Train the model with validation
train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=20)

# Evaluation
model.eval()
correct_predictions = 0
total_predictions = 0
all_labels = []
all_predictions = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        all_labels.extend(labels.numpy())
        all_predictions.extend(predicted.numpy())

accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy:.4f}")
accuracy = correct_predictions / total_predictions


In [ ]:
# Confusion matrix and classification report
confusion_mat = confusion_matrix(all_labels, all_predictions)
print("Confusion Matrix:")
print(confusion_mat)

print("\nClassification Report:")
print(classification_report(all_labels, all_predictions))

# Visualize some predictions
for i in range(min(5, len(all_labels))):  # Ensure that we don't exceed the length of the lists
    print(f"True Label: {all_labels[i]}, Predicted Label: {all_predictions[i]}")
